# Monefy ETL
El objetivo de este notebook es extraer, limpiar, procesar, categorizar y guardar en una base de SQLite listas para visualizar en Notion las transacciones relacionadas con mis gastos e ingresos financieros registrados en la app de Monefy.
El proceso se lleva a cabo de la siguiente manera:
1. Exportar .csv desde la app de Monefy
2. Guardar achivo en carpeta base de Google Drive como **"Monefy.Data.csv"**
3. Este notebook se ejecutá automaticamente todos los dias a las 7:00 am para extraer, ordenar, limpiar y guardar en SQLite los datos para presentarlos en un dashboard en Notion

### For Deepnote

In [1]:
!ls /datasets/google-drive
!cd /datasets/google-drive


'Boardgames and D&D'		   Fotos
'BUSQUEDA RENTAS QUERETARO.xlsx'  'Monefy (5b65920e).Data.csv'
 Cocoestudio			   Monefy.Data.csv
'Colab Notebooks'		   monthly_budget.ipynb
 Data				  'Presupuesto mensual.desktop'
 Documentos			   Untitled0.ipynb


### For Google Colab

In [2]:
# !pip install google.colab
# from google.colab import drive
# drive.mount('/content/gdrive')
# %cd /content/gdrive/MyDrive/Documentos/Finanzas/Monefy

## Lectura y almacenamiento de datos brutos (Raw)

In [3]:
import sqlite3
import numpy as np
from datetime import datetime
from sqlalchemy import create_engine
import pandas as pd

#DATABASE_LOCATION = "sqlite:////content/gdrive/My Drive/Documentos/Finanzas/Monefy/monefy"
DATABASE_LOCATION = "sqlite://///datasets/google-drive/monefy"
data = pd.read_csv('/datasets/google-drive/Monefy (5b65920e).Data.csv')

engine = create_engine(DATABASE_LOCATION)
conn = sqlite3.connect('monefy.sqlite')
print('Opened database succesfully')
cursor = conn.cursor()
q1 = "DROP TABLE IF EXISTS raw_monefy_records;"
q2 =  """
    CREATE TABLE IF NOT EXISTS raw_monefy_records(
        date varchar(30) NOT NULL,
        account varchar(30) NOT NULL,
        category varchar(60) NOT NULL,
        amount int NOT NULL,
        currency varchar(3) NOT NULL,
        converted_amount int NOT NULL,
        currency_conv varchar(3) NOT NULL,
        description varchar(100) NULL
    )
    """
cursor.execute(q1)
print('Droped table raw_monefy_records succesfully')
cursor.execute(q2)
print('New data written on raw_monefy_records succesfully')
conn.commit()

# Populate empty table
data.to_sql('raw_monefy_records',conn, index = False, if_exists= 'replace')
conn.close()
print('Loaded data. Database closed succesfully')

Opened database succesfully
Droped table raw_monefy_records succesfully
New data written on raw_monefy_records succesfully
Loaded data. Database closed succesfully
/shared-libs/python3.9/py/lib/python3.9/site-packages/pandas/core/generic.py:2779: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  sql.to_sql(


## Funciones para transformación y validación de datos

In [4]:
def check_if_empty(df: pd.DataFrame) -> bool:
  #Check for empty dataframe
  if df.empty:
    raise Exception('Empty dataframe. Please verify export file! Finishing execution')
    return False

def clean_data(df: pd.DataFrame) -> bool:
  #Clean NULLs
  df = df.replace(np.nan,'',regex= True)
  if df.isnull().values.any():
    raise Exception('Nulls found. Please verify export file! Finishing execution')
    return False
  else:
    return df

def rm_transfers(df: pd.DataFrame):
  index = df.loc[df['category'].str.startswith('To ', na = False)].index
  df.drop(index,inplace = True)
  index = df.loc[df['category'].str.startswith('From ', na = False)].index
  df.drop(index,inplace = True)
  index = df.loc[df['category'].str.startswith('Initial ', na = False)].index
  df.drop(index,inplace = True)
  return df

def add_date(df: pd.DataFrame):
  df['date'] = pd.to_datetime(df['date'])
  df['year'] = df['date'].dt.year
  df['month'] = df['date'].dt.month
  df['day'] = df['date'].dt.day
  return df

def set_values(row, value):
    return value[row]

def categories(df):
  df['category_name'] = df['category'].apply(set_values, args =(cat_dict,))
  return df

def transaction_id(df: pd.DataFrame):
  pass

In [5]:
if check_if_empty(data):
    raise Exception('Check for empty data')
else:
    print('Data not empty. Proceed to transform...')
    data = add_date(rm_transfers(clean_data(data)))
    data.rename(columns={'converted amount':'conv_amount','currency.1':'conv_currency'},inplace=True)
    data['amount'] = data['amount'].str.replace(',','')
    data['amount'] = data['amount'].astype(float)
    data['conv_amount'] = data['conv_amount'].str.replace(',','')
    data['conv_amount'] = data['conv_amount'].astype(float)
    print('Data transformed succesfully')

Data not empty. Proceed to transform...
Data transformed succesfully


In [6]:


df_3 = _deepnote_execute_sql("""select * from data
where year = 2022
and month = 9
""", 'SQL_DEEPNOTE_DATAFRAME_SQL')
df_3

,date,account,category,amount,currency,conv_amount,conv_currency,description,year,month,day
0,2022-09-03,Nu,Misceláneos,-149.00,MXN,-149.00,MXN,Spotify,2022,9,3
1,2022-09-06,HSBC,Sueldo,12790.94,MXN,12790.94,MXN,Nómina,2022,9,6
2,2022-09-07,Nu,Despensa,-1034.90,MXN,-1034.90,MXN,Chedraui,2022,9,7
3,2022-09-07,Nu,Despensa,-355.00,MXN,-355.00,MXN,Costco,2022,9,7
4,2022-09-08,Nu,Gustos,-370.00,MXN,-370.00,MXN,Italian Coffee ☕,2022,9,8
5,2022-09-08,Nu,Cuidado personal,-3118.00,MXN,-3118.00,MXN,Patines y protecciones,2022,9,8
6,2022-09-08,Nu,Cuidado personal,-229.00,MXN,-229.00,MXN,Leggins,2022,9,8
7,2022-09-15,HSBC,Sueldo,12790.94,MXN,12790.94,MXN,Nómina,2022,9,15
8,2022-09-20,Nu,Hogar,-333.00,MXN,-333.00,MXN,Pago Nokia,2022,9,20
9,2022-09-29,HSBC,Sueldo,12790.94,MXN,12790.94,MXN,Nómina,2022,9,29


conn = sqlite3.connect('monefy.sqlite')
cursor = conn.cursor()
q1 = "DROP TABLE IF EXISTS curated_records;"
q2 =  """
    CREATE TABLE IF NOT EXISTS curated_records(
        date varchar(30) NOT NULL
        ,account varchar(30) NOT NULL
        ,category varchar(60) NOT NULL
        ,amount int NOT NULL
        ,currency varchar(3) NOT NULL
        ,converted_amount int NOT NULL
        ,currency_conv varchar(3) NOT NULL
        ,description varchar(100) NULL
        ,year int NOT NULL
        ,month int NOT NULL
        ,day int NOT NULL
    )
    """
cursor.execute(q1)
cursor.execute(q2)
conn.commit()

# Populate empty table
data.to_sql('curated_records',conn, schema= None, index = False, if_exists= 'append')
conn.close()
print('Loaded data. Database closed succesfully')

In [7]:
conn = sqlite3.connect('monefy.sqlite')
cur_data = pd.read_sql_query("SELECT * from curated_records", conn)
conn.close()

# Carga de API para Notion


In [8]:
import os
import requests
import pandas as pd

# keys are stored in env vars to be hidden from users
api_key = "secret_qgYbeHTb3dvMtesG7uwqJJZYwba0QM2WopTenYe5Eih" #os.environ["NOTION_API_KEY"]
database_id = "d6a91f43783e45c4a81541c8030393ae" #os.environ["DATABASE_ID"]



In [9]:
from IPython.display import display, JSON

headers = {
    "Authorization": f"Bearer {api_key}",
    "Notion-Version": "2021-08-16",
    "Content-Type": "application/json",
}

# load first page
response = requests.post(
    f"https://api.notion.com/v1/databases/{database_id}/query", headers=headers
).json()

print("First result:")
display(JSON(response))

# iteratively load all pages
records = response["results"]
while response["has_more"]:
    response = requests.post(
        f"https://api.notion.com/v1/databases/{database_id}/query",
        json={"start_cursor": response["next_cursor"]},
        headers=headers,
    ).json()
    print("Loading page ...")
    records.extend(response.get("results", []))

print(f"Got {len(records)} entries from 'Presupuesto Mensual' in Notion.")
print("Records have these properties", [x for x in records[0]['properties']])

First result:


<IPython.core.display.JSON object>

Got 26 entries from 'Presupuesto Mensual' in Notion.
Records have these properties ['Automatico', 'Categoria', 'Type', 'Presupuesto', 'Description']


### Create DataFrame from Notion data

In [10]:
def get_raw_value(item):
    item_type = item['type']
    if type(item[item_type]) is list:
        if item[item_type][0]['type'] == 'text':
            return item[item_type][0]['plain_text']
    if item_type == 'select':
        return item[item_type]['name']
    return item[item_type]

all_values = []   
for record in records:
    properties = record['properties']
    all_values.append({
        'Description': get_raw_value(properties['Description']),
        'Type': get_raw_value(properties['Type']),
        'Categoria': get_raw_value(properties['Categoria']),
        'Presupuesto': get_raw_value(properties['Presupuesto']),
        'Automatico': get_raw_value(properties['Automatico'])
    })

df = pd.DataFrame(all_values)
df.fillna(0,inplace=True)

In [11]:


budget = _deepnote_execute_sql("""SELECT
    description,
    type,
    categoria,
    presupuesto as presupuesto_quincenal,
    (presupuesto * 2) as presupuesto_mensual,
    automatico,
    month,
    year
FROM df
CROSS JOIN (
    SELECT month(now()) as month ,year(now()) as year
)
""", 'SQL_DEEPNOTE_DATAFRAME_SQL')
budget

,Description,Type,Categoria,presupuesto_quincenal,presupuesto_mensual,Automatico,month,year
0,Vales,Income,Sueldo,500.0,1000.0,False,2,2023
1,Adicionales,Income,Adicionales,0.0,0.0,False,2,2023
2,Proyectos,Expense,Hogar,0.0,0.0,False,2,2023
3,Salud,Expense,Cuidado personal,0.0,0.0,False,2,2023
4,Ropa,Expense,Cuidado personal,0.0,0.0,False,2,2023
5,Regalos,Expense,Miscenláneos,0.0,0.0,False,2,2023
6,Entretenimiento,Expense,Gustos,0.0,0.0,False,2,2023
7,Calle,Expense,Miscenláneos,0.0,0.0,False,2,2023
8,Antojos,Expense,Gustos,0.0,0.0,False,2,2023
9,Saldo,Expense,Hogar,-100.0,-200.0,False,2,2023


In [12]:


filtered_data = _deepnote_execute_sql("""SELECT
category
,sum(CAST (amount as int)) as monto_real
FROM data
WHERE 
    year = year(now())
    AND month = month(now())
GROUP BY
category
""", 'SQL_DEEPNOTE_DATAFRAME_SQL')
filtered_data

,category,monto_real
0,Sueldo,38370
1,Hogar,-5333


In [13]:


presup_vs_real = _deepnote_execute_sql("""SELECT 
    a.categoria as Categoria,
    a.description as Description,
    coalesce(sum(a.presupuesto_mensual),0) as "Monto presupuesto",
    coalesce(sum(b.monto_real),0) as "Monto real",
    coalesce(sum(a.presupuesto_mensual) - sum(b.monto_real),0) as "Diferencia",
    a.month as mes_actual,
    a.year as año_actual
FROM budget a
LEFT JOIN filtered_data b
    ON a.categoria = b.category
GROUP BY 
    a.categoria,
    a.description,
    a.month,
    a.year
""", 'SQL_DEEPNOTE_DATAFRAME_SQL')
presup_vs_real

,Categoria,Description,Monto presupuesto,Monto real,Diferencia,mes_actual,año_actual
0,Sueldo,Vales,1000.0,38370,-37370.0,2,2023
1,Hogar,Proyectos,0.0,-5333,5333.0,2,2023
2,Hogar,Saldo,-200.0,-5333,5133.0,2,2023
3,Hogar,Renta,-8100.0,-5333,-2767.0,2,2023
4,Sueldo,Sueldo,24400.0,38370,-13970.0,2,2023
5,Adicionales,Adicionales,0.0,0,0.0,2,2023
6,Cuidado personal,Salud,0.0,0,0.0,2,2023
7,Cuidado personal,Ropa,0.0,0,0.0,2,2023
8,Miscenláneos,Regalos,0.0,0,0.0,2,2023
9,Gustos,Entretenimiento,0.0,0,0.0,2,2023


In [31]:
DeepnoteChart(presup_vs_real, """{"layer":[{"layer":[{"mark":{"clip":true,"type":"bar","color":"#4c78a8","tooltip":true},"encoding":{"x":{"sort":null,"type":"nominal","field":"Categoria","scale":{"type":"linear"}},"y":{"sort":null,"type":"quantitative","field":"Monto real","scale":{"type":"linear"},"format":{"type":"default","decimals":null},"formatType":"numberFormatFromNumberType"},"color":{"datum":"Monto real"},"xOffset":{"datum":"Monto real"}}}]},{"layer":[{"mark":{"clip":true,"type":"bar","color":"#f58518","tooltip":true},"encoding":{"x":{"sort":null,"type":"nominal","field":"Categoria","scale":{"type":"linear"}},"y":{"sort":null,"type":"quantitative","field":"Monto presupuesto","scale":{"type":"linear"},"format":{"type":"default","decimals":null},"formatType":"numberFormatFromNumberType"},"color":{"datum":"Monto presupuesto"},"xOffset":{"datum":"Monto presupuesto"}}}]}],"title":"Por Categoria","config":{"legend":{}},"$schema":"https://vega.github.io/schema/vega-lite/v5.json","encoding":{}}""")

In [37]:
DeepnoteChart(presup_vs_real, """{"layer":[{"layer":[{"mark":{"clip":true,"type":"bar","color":"#4c78a8","tooltip":true},"encoding":{"x":{"sort":null,"type":"nominal","field":"Description","scale":{"type":"linear"}},"y":{"sort":null,"type":"quantitative","field":"Monto real","scale":{"type":"linear"},"format":{"type":"default","decimals":null},"formatType":"numberFormatFromNumberType"},"color":{"datum":"Monto real"},"xOffset":{"datum":"Monto real"}}}]},{"layer":[{"mark":{"clip":true,"type":"bar","color":"#f58518","tooltip":true},"encoding":{"x":{"sort":null,"type":"nominal","field":"Description","scale":{"type":"linear"}},"y":{"sort":null,"type":"quantitative","field":"Monto presupuesto","scale":{"type":"linear"},"format":{"type":"default","decimals":null},"formatType":"numberFormatFromNumberType"},"color":{"datum":"Monto presupuesto"},"xOffset":{"datum":"Monto presupuesto"}}}]}],"title":"Por Descripción","config":{"legend":{}},"$schema":"https://vega.github.io/schema/vega-lite/v5.json","encoding":{}}""")

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=32110792-5b0d-478b-8e48-fe25fdadce3a' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>